This notebook prepares the previously retrieved Quarry data for analysis, along with some basic exploratory analysis.

In [199]:
import pandas as pd
import numpy as np
import geoip2.database
import geocoder
from p5utils import *

[Quarry](https://meta.wikimedia.org/wiki/Research:Quarry) is a web interface for requesting data from Wiki Replicas, a set of databases which replicate, in real time, the actual production MariaSQL dbs that all Wikimedia projects run on. All queries made on Quarry can be publicly viewed: [here's mine](https://quarry.wmflabs.org/query/36617), which generated the .tsv file, also reproduced here:

```sql
use enwiki_p;

SELECT rc_timestamp, rc_actor, rc_title, rc_this_oldid, rc_old_len, rc_new_len, actor_name
	FROM recentchanges
		LEFT JOIN actor ON rc_actor = actor_id
			WHERE rc_type = 0
			AND rc_namespace = 0
			AND actor_user IS null
LIMIT 1000000;
```

In [2]:
changes = pd.read_csv('anon_changes_jun04.tsv', sep='\t')
changes['len_diff'] = changes['rc_new_len'] - changes['rc_old_len']
changes = changes.query("rc_actor != 0.0")
changes = changes.reset_index()

The damage_chunks data comes from running `get_damage_probs.py` in 1,000-chunk increments simultaneously in four separate terminals (aka "manual async"). Each chunk contains the ORES damage probability score for 50 or 51 revisions. Gathering this data for approximately 730,000 revisions took several hours.

In [3]:
damage_chunks = []
for i in range(14588):
    damage_chunks.append(pd.read_csv(f'./damage_chunks/damage_chunk_{i}.csv',
                             usecols=[1], header=None, squeeze=True))
damage_prob = pd.concat(damage_chunks, ignore_index=True)

In [6]:
damage_prob.to_csv('damage_prob.csv', header=False)

In [7]:
changes['damage_prob'] = damage_prob

Some of the damage probabilities are NaN. These are all revisions that were completely deleted by a Wikipedia administrator according to the [revision deletion policy](https://en.wikipedia.org/wiki/Wikipedia:Revision_deletion), where revisions that consist of copyright violations, libel, grossly offensive content, etc. are fully deleted from the page history (rather than simply reverted, like less severe vandalism). I rescored these as 1.0 (i.e., a 100% chance of being damaging).

In [17]:
changes['damage_prob'] = changes['damage_prob'].fillna(value=1.0)

In [18]:
unique_ips = changes['actor_name'].unique()

In [20]:
def get_geoinfo(ip_address, reader):
    r = reader.city(ip_address)
    return [r.location.latitude,
            r.location.longitude,
            r.city.name,
            r.subdivisions.most_specific.name,
            r.country.name,
            r.country.iso_code,
            r.postal.code]

After investigating several IP geocoding services (all of which charged quite a bit of money for coding more than a few thousand IPs, much less several hundred thousand), I found the [GeoLite2 database](https://dev.maxmind.com/geoip/geoip2/geolite2/) offered for free by MaxMind in order to promote their more accurate GeoIP2 data. GeoLite2 is less accurate (the lat/long coordinates are centered in their respective postcodes/cities), but perfectly good for what I needed, free, and (importantly) very fast, since it didn't need to be accessed through a web API.

The geoip2 package they offered made coding almost too easy.

In [21]:
reader = geoip2.database.Reader('./GeoLite2-City_20190604/GeoLite2-City.mmdb')
locs = []
for ip in unique_ips:
    locs.append(get_geoinfo(ip, reader))

In [22]:
ip_locations = pd.concat([pd.Series(unique_ips), (pd.DataFrame(locs))], axis=1)
ip_locations.columns = [
    'actor_name', 'lat', 'lng', 'city', 'state', 'country', 'country_iso', 'postcode'
]

In [33]:
changes = pd.merge(changes, ip_locations, on='actor_name').drop('index', axis=1)

I noticed some NaN values in my article title column, which was unexpected. The cause: several revisions in my dataset were to the [Wikipedia article on the concept of NaN](https://en.wikipedia.org/wiki/NaN), which the Pandas data import coded as literal NaN values!

In [195]:
changes['rc_title'] = changes['rc_title'].fillna('NaN')

More NA resolving - the GeoLite2 database whiffed on some IPs, so I used the geocoder package to clean up the stragglers. Geocoder uses IPInfo under the hood, which has a 1,000/day rate limit for free access - more than enough.

In [215]:
for ip in changes[changes['lat'].isna()]['actor_name']:
    g = geocoder.ip(ip)
    changes.loc[changes['actor_name']==ip, 'lat'] = g.lat
    changes.loc[changes['actor_name']==ip, 'lng'] = g.lng
    changes.loc[changes['actor_name']==ip, 'city'] = g.city
    changes.loc[changes['actor_name']==ip, 'state'] = g.state
    changes.loc[changes['actor_name']==ip, 'country'] = 'United States'
    changes.loc[changes['actor_name']==ip, 'country_iso'] = g.country

Create a combined lat/long column (for intended use in Google Earth Engine, but I ended up not going that route) and save our dataset to .csv for later use.

In [226]:
changes['latlng'] = changes.apply(lambda x: (x['lat'], x['lng']), axis=1)

In [228]:
changes.to_csv('changes.csv')

Filter out a US-only version to play around with.

In [245]:
changes_us = changes[changes['country_iso']=='US']

Let's see what people in my city and ZIP code are editing.

In [142]:
np.random.choice(changes[changes['city']=='Chicago']['rc_title'].unique(), size=5)

array(['Edwardsville,_Illinois', 'My_Bloody_Valentine_(band)',
       'Christine_(name)', 'Probation', 'Shepley,_Rutan_and_Coolidge'],
      dtype=object)

In [143]:
changes[changes['postcode']=='60647']['rc_title'].value_counts().head()

Tom_Cat                                          16
Tom_and_Jerry:_Robin_Hood_and_His_Merry_Mouse     9
Baby_Puss                                         7
Varsity_(band)                                    6
Song_Review:_A_Greatest_Hits_Collection           5
Name: rc_title, dtype: int64

Uh, weird. What about the top edited articles in the entire dataset?

In [130]:
changes['rc_title'].value_counts().head()

List_of_Produce_X_101_contestants    383
2019_Indian_general_election         377
Kenzō_Shirai                         356
2019–20_UEFA_Champions_League        343
Aladdin_(2019_film)                  325
Name: rc_title, dtype: int64

Here's what the deleted/added text fields look like.

In [157]:
get_changed_text(900103915)

Deleted:
 []
Added:
 ['', '===Claims===', "Journalist, scholar and author [[Toby Lester]] summarizes the  claims of Hagarism's at odds with Islamic doctrine (followed by quotes of the authors in parenthesis):", '*"suggestions that the text of the Quran came into being later than is now believed" ("There is no hard evidence for the existence of the Koran in any form before the last decade of the seventh century"); ', '*"that [[Mecca]] was not the initial Islamic sanctuary" ("[the evidence] points unambiguously to a sanctuary in north-west Arabia ... Mecca was secondary"); ', '*"that the Arab conquests preceded the institutionalization of Islam ("the Jewish messianic fantasy was enacted in the form of an Arab conquest of the Holy Land"); ', '*"that the idea of the [[Hegira|hijra]], or the migration of Muhammad and his followers from Mecca to [[Medina]] in 622, may have evolved long after Muhammad died" ("No seventh-century source identifies the Arab era as that of the hijra"); and ', '*"

/Users/aaron/metis/project-05/p5utils.py:7: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 7 of the file /Users/aaron/metis/project-05/p5utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(response['compare']['*'], features='lxml')


Most vandalous countries are likely just variance due to very small N.

In [273]:
changes.groupby('country')['damage_prob'].mean().sort_values(ascending=False).head()

country
French Guiana                        0.953278
Bonaire, Sint Eustatius, and Saba    0.739908
Gabon                                0.685797
Anguilla                             0.681811
Comoros                              0.671198
Name: damage_prob, dtype: float64

In [274]:
changes_us.groupby('state')['damage_prob'].mean().sort_values(ascending=False).head()

state
Wyoming          0.540221
Iowa             0.493879
New Hampshire    0.487237
Delaware         0.475978
Mississippi      0.473873
Name: damage_prob, dtype: float64